가장 간단한 한국어 자연어 처리모델
LSTM 텍스트 모델에 효과가 좋다.

In [134]:
# 벡터를 이용한 모델을 생성해서 문자열 값을 리턴 받아보자.
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import tensorflow as tf
import numpy as np


In [135]:
#1. Naver Sentiment Movie Corpus v1.0 다운로드
train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
test_file = tf.keras.utils.get_file('test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

In [136]:
#파일을 이진모드로 읽어온다. 디코드는 utf8로한다.
text_train = open(train_file,'rb').read().decode(encoding='utf-8')
text_test = open(test_file,'rb').read().decode(encoding='utf-8')

In [137]:
a=text_test[:500].split('\n')[1:]
# print(a[5])
print(a[3].count('\t'))
a[5]


2


'7898805\t음악이 주가 된, 최고의 음악영화\t1'

In [138]:
a[5].split('\t') [2]  # 감상평의 평점

'1'

In [139]:
print(text_test[:200])

#각문장을 \n으로 분리하겠다.
#헤더를 제외 \t하겠다. 
#문자를 임베딩해서 2차원으로 변환하겠다.

#한줄마다 \t로 분할해서 정수화?
#감상평은 \t로 구분되어있음
# [1:]으로 헤더제거
#감상평의 평점만 뽑아서 정수화
y_train= np.array([ [int(row.split('\t')[2])] for row in text_train.split('\n')[1:] if row.count('\t')>0])
y_test= np.array([ [int(row.split('\t')[2])] for row in text_test.split('\n')[1:] if row.count('\t')>0])
print(y_train.shape) # (150000개의 평점 , 1개의 열)


id	document	label
6270596	굳 ㅋ	1
9274899	GDNTOPCLASSINTHECLUB	0
8544678	뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아	0
6825595	지루하지는 않은데 완전 막장임... 돈주고 보기에는....	0
6723715	3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심
(150000, 1)


In [140]:
text_train.split('\n')  #한줄씩 분리하고
text_train.split('\n')[1:]  #헤더 처리
#row.count('\t')>0   평점이 존재하면
# row.split('\t')[1]  감상평만 추출

['9976970\t아 더빙.. 진짜 짜증나네요 목소리\t0',
 '3819312\t흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나\t1',
 '10265843\t너무재밓었다그래서보는것을추천한다\t0',
 '9045019\t교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정\t0',
 '6483659\t사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다\t1',
 '5403919\t막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.\t0',
 '7797314\t원작의 긴장감을 제대로 살려내지못했다.\t0',
 '9443947\t별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네\t0',
 '7156791\t액션이 없는데도 재미 있는 몇안되는 영화\t1',
 '5912145\t왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?\t1',
 '9008700\t걍인피니트가짱이다.진짜짱이다♥\t1',
 '10217543\t볼때마다 눈물나서 죽겠다90년대의 향수자극!!허진호는 감성절제멜로의 달인이다~\t1',
 '5957425\t울면서 손들고 횡단보도 건널때 뛰쳐나올뻔 이범수 연기 드럽게못해\t0',
 '8628627\t담백하고 깔끔해서 좋다. 신문기사로만 보다 보면 자꾸 잊어버린다. 그들도 사람이었다는 것을.\t1',
 '9864035\t취향은 존중한다지만 진짜 내생에 극장에서 본 영화중 가장 노잼 노감동임 스토리도 어거지고 감동도 어거지\t0',
 '6852435\tㄱ냥 매번 긴장되고 재밋음ㅠㅠ\t1',
 '9143163\t참 사람들 웃긴게 바스코가 이기면 락스코라고 까고바비가 이기면 아이돌이라고 깐다.그냥 까고싶어서 안달난것처럼 보인다\t1',
 '4891476\t굿바이 레닌 표절인것은 이해하는데 왜 뒤로 갈수록 재미없어지냐\t0'

In [149]:
#x 값 추출 

x_text_train = [row.split('\t')[1] for row in text_train.split('\n')[1:] if row .count('\t')>0  ]
x_text_train[1]


'흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'

In [150]:
import re 
#뽑아낸 문장에서 필요없는 것들을 제거 해줄수있다.
#물음표만 제가해본다.
#점도 제거
def clean_str(string): 
    string =re.sub(r'\?','',string)
    string =re.sub(r'\.','',string)
    # string =re.sub(r'는','',string)
    # string =re.sub(r'이','',string)
    # string =re.sub(r'가','',string)
    return string.lower()
a= '나는 너가 행복하길 바라'
# clean_str(x_text_train[1])
x_text_train= [clean_str(s) for s in x_text_train ]

#필요없는것 처리후 스페이스를 기준으로 단어(형태소?)분리
x_text_train= [ s.split(' ') for s in x_text_train]
x_text_train

#문장마다 가진 단어의 갯수를 확인 => 생략
#너무 문장이 길면 생략하는 선택지도 있다.


[['아', '더빙', '진짜', '짜증나네요', '목소리'],
 ['흠포스터보고', '초딩영화줄오버연기조차', '가볍지', '않구나'],
 ['너무재밓었다그래서보는것을추천한다'],
 ['교도소', '이야기구먼', '솔직히', '재미는', '없다평점', '조정'],
 ['사이몬페그의',
  '익살스런',
  '연기가',
  '돋보였던',
  '영화!스파이더맨에서',
  '늙어보이기만',
  '했던',
  '커스틴',
  '던스트가',
  '너무나도',
  '이뻐보였다'],
 ['막', '걸음마', '뗀', '3세부터', '초등학교', '1학년생인', '8살용영화ㅋㅋㅋ별반개도', '아까움'],
 ['원작의', '긴장감을', '제대로', '살려내지못했다'],
 ['별',
  '반개도',
  '아깝다',
  '욕나온다',
  '이응경',
  '길용우',
  '연기생활이몇년인지정말',
  '발로해도',
  '그것보단',
  '낫겟다',
  '납치감금만반복반복이드라마는',
  '가족도없다',
  '연기못하는사람만모엿네'],
 ['액션이', '없는데도', '재미', '있는', '몇안되는', '영화'],
 ['왜케', '평점이', '낮은건데', '꽤', '볼만한데', '헐리우드식', '화려함에만', '너무', '길들여져', '있나'],
 ['걍인피니트가짱이다진짜짱이다♥'],
 ['볼때마다', '눈물나서', '죽겠다90년대의', '향수자극!!허진호는', '감성절제멜로의', '달인이다~'],
 ['울면서', '손들고', '횡단보도', '건널때', '뛰쳐나올뻔', '이범수', '연기', '드럽게못해'],
 ['담백하고',
  '깔끔해서',
  '좋다',
  '신문기사로만',
  '보다',
  '보면',
  '자꾸',
  '잊어버린다',
  '그들도',
  '사람이었다는',
  '것을'],
 ['취향은',
  '존중한다지만',
  '진짜',
  '내생에',
  '극장에서',
  '본',
  '영화중',
  '가장',
  '노잼',
  '노감동임',
  '스토리도',
  '어거지고

In [151]:
sentences=x_text_train

In [163]:
#tokenizer 와 pad_sequences를 사용한 문장 전처리
#텐서 플로우에있는 토큰화모듈을 이용해본다.
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#빈도수가 높은 단어들만 추려서 숫자형으로 변환한다.
tokenizer =Tokenizer(num_words=2000) #빈도수 순위 2000까지만
tokenizer.fit_on_texts(sentences) #단어를 분리한 문장들 토큰화한다? none값나옴 먼지모름 알아볼것

#단어들을 빈도 순위대로 정수화한것이다.
sequences =tokenizer.texts_to_sequences(sentences)
print(sequences[:5])

x_train =pad_sequences(sequences,padding='post')
x_train[:5]  

#시퀀스 길이를 동일한 25칸까지 패딩속성=post(pre하면 앞공간)로 뒷 빈공간을 0으로 채워졌다.

[[43, 901, 4, 1069], [], [], [71, 336], [98, 790, 523]]


array([[  43,  901,    4, 1069,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [  71,  336,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0],
       [  98,  790,  523,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        